Initial Data Exploration

In [1]:
import pandas as pd

In [2]:
raw_data_2022_mn_pm25 = "Data/2022_MN_PM_25.csv"

In [3]:
df_raw_2022_mn_pm25 = pd.read_csv(raw_data_2022_mn_pm25)

In [4]:
df_raw_2022_mn_pm25

,Date,Source,Site ID,POC,Daily Mean PM2.5 Concentration,UNITS,DAILY_AQI_VALUE,Site Name,DAILY_OBS_COUNT,PERCENT_COMPLETE,AQS_PARAMETER_CODE,AQS_PARAMETER_DESC,CBSA_CODE,CBSA_NAME,STATE_CODE,STATE,COUNTY_CODE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE
0,01/02/2022,AQS,270031002,1,13.5,ug/m3 LC,54,Anoka County Airport,1,100.0,88101,PM2.5 - Local Conditions,33460.0,"Minneapolis-St. Paul-Bloomington, MN-WI",27,Minnesota,3,Anoka,45.13768,-93.207615
1,01/05/2022,AQS,270031002,1,3.9,ug/m3 LC,16,Anoka County Airport,1,100.0,88101,PM2.5 - Local Conditions,33460.0,"Minneapolis-St. Paul-Bloomington, MN-WI",27,Minnesota,3,Anoka,45.13768,-93.207615
2,01/08/2022,AQS,270031002,1,9.6,ug/m3 LC,40,Anoka County Airport,1,100.0,88101,PM2.5 - Local Conditions,33460.0,"Minneapolis-St. Paul-Bloomington, MN-WI",27,Minnesota,3,Anoka,45.13768,-93.207615
3,01/11/2022,AQS,270031002,1,6.7,ug/m3 LC,28,Anoka County Airport,1,100.0,88101,PM2.5 - Local Conditions,33460.0,"Minneapolis-St. Paul-Bloomington, MN-WI",27,Minnesota,3,Anoka,45.13768,-93.207615
4,01/14/2022,AQS,270031002,1,2.6,ug/m3 LC,11,Anoka County Airport,1,100.0,88101,PM2.5 - Local Conditions,33460.0,"Minneapolis-St. Paul-Bloomington, MN-WI",27,Minnesota,3,Anoka,45.13768,-93.207615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8547,12/25/2022,AQS,271713201,3,3.6,ug/m3 LC,15,St. Michael Elementary School,1,100.0,88101,PM2.5 - Local Conditions,33460.0,"Minneapolis-St. Paul-Bloomington, MN-WI",27,Minnesota,171,Wright,45.20916,-93.669210
8548,12/26/2022,AQS,271713201,3,3.6,ug/m3 LC,15,St. Michael Elementary School,1,100.0,88101,PM2.5 - Local Conditions,33460.0,"Minneapolis-St. Paul-Bloomington, MN-WI",27,Minnesota,171,Wright,45.20916,-93.669210
8549,12/27/2022,AQS,271713201,3,5.0,ug/m3 LC,21,St. Michael Elementary School,1,100.0,88101,PM2.5 - Local Conditions,33460.0,"Minneapolis-St. Paul-Bloomington, MN-WI",27,Minnesota,171,Wright,45.20916,-93.669210
8550,12/28/2022,AQS,271713201,3,8.7,ug/m3 LC,36,St. Michael Elementary School,1,100.0,88101,PM2.5 - Local Conditions,33460.0,"Minneapolis-St. Paul-Bloomington, MN-WI",27,Minnesota,171,Wright,45.20916,-93.669210


In [5]:
df_raw_2022_mn_pm25.nunique()

Date                              365
Source                              2
Site ID                            25
POC                                 4
Daily Mean PM2.5 Concentration    275
UNITS                               1
DAILY_AQI_VALUE                    93
Site Name                          25
DAILY_OBS_COUNT                     1
PERCENT_COMPLETE                    1
AQS_PARAMETER_CODE                  2
AQS_PARAMETER_DESC                  2
CBSA_CODE                           8
CBSA_NAME                           8
STATE_CODE                          1
STATE                               1
COUNTY_CODE                        18
COUNTY                             18
SITE_LATITUDE                      25
SITE_LONGITUDE                     25
dtype: int64

In [18]:
df_raw_2022_mn_pm25_dropped = df_raw_2022_mn_pm25.drop(columns = ["Source", "Site ID", "DAILY_OBS_COUNT", "PERCENT_COMPLETE", 
                                                                  "AQS_PARAMETER_CODE", "AQS_PARAMETER_DESC", "CBSA_CODE", 
                                                                  "CBSA_NAME", "COUNTY_CODE", "STATE_CODE", "STATE"])
df_raw_2022_mn_pm25_dropped

,Date,POC,Daily Mean PM2.5 Concentration,UNITS,DAILY_AQI_VALUE,Site Name,COUNTY,SITE_LATITUDE,SITE_LONGITUDE
0,01/02/2022,1,13.5,ug/m3 LC,54,Anoka County Airport,Anoka,45.13768,-93.207615
1,01/05/2022,1,3.9,ug/m3 LC,16,Anoka County Airport,Anoka,45.13768,-93.207615
2,01/08/2022,1,9.6,ug/m3 LC,40,Anoka County Airport,Anoka,45.13768,-93.207615
3,01/11/2022,1,6.7,ug/m3 LC,28,Anoka County Airport,Anoka,45.13768,-93.207615
4,01/14/2022,1,2.6,ug/m3 LC,11,Anoka County Airport,Anoka,45.13768,-93.207615
...,...,...,...,...,...,...,...,...,...
8547,12/25/2022,3,3.6,ug/m3 LC,15,St. Michael Elementary School,Wright,45.20916,-93.669210
8548,12/26/2022,3,3.6,ug/m3 LC,15,St. Michael Elementary School,Wright,45.20916,-93.669210
8549,12/27/2022,3,5.0,ug/m3 LC,21,St. Michael Elementary School,Wright,45.20916,-93.669210
8550,12/28/2022,3,8.7,ug/m3 LC,36,St. Michael Elementary School,Wright,45.20916,-93.669210


In [7]:
df_raw_2022_mn_pm25["Date"].value_counts()

05/29/2022    32
05/05/2022    32
01/17/2022    32
02/22/2022    32
09/02/2022    32
              ..
05/13/2022    17
07/14/2022    16
07/15/2022    14
07/17/2022    14
07/18/2022    14
Name: Date, Length: 365, dtype: int64

In [6]:
df_raw_2022_mn_pm25["Site Name"].value_counts()

Harding High School                     801
Apple Valley                            491
Anoka County Airport                    468
Andersen School                         458
Boundary Waters                         413
Southwest Minnesota Regional Airport    362
Ben Franklin School                     359
Leech Lake Nation:                      358
Fond du Lac Band                        356
Near Road I-35                          355
B.F. Pearson School                     355
Red Lake Nation                         355
Near Road I-35/I-94                     353
St. Michael Elementary School           349
U of M - Duluth                         348
Brainerd Lakes Regional Airport         343
FWS Wetland Management District         338
Grand Portage Band                      335
Virginia City Hall                      331
Laura MacArthur School                  312
Talahi School                           277
St. Louis Park City Hall                145
Ramsey Health Center            

In [19]:
df_raw_2022_mn_pm25_dropped.columns

Index(['Date', 'POC', 'Daily Mean PM2.5 Concentration', 'UNITS',
       'DAILY_AQI_VALUE', 'Site Name', 'COUNTY', 'SITE_LATITUDE',
       'SITE_LONGITUDE'],
      dtype='object')

In [21]:
columns = {"Date": "Date", "POC": "POC", "Daily Mean PM2.5 Concentration": "Day_Mean_PM25_Conc", "UNITS": "Units", 
           "DAILY_AQI_VALUE":"Daily_AQI", "Site Name": "Site_Name", "COUNTY": "County", "SITE_LATITUDE": "Latitude",
          "SITE_LONGITUDE": "Longitude"}

In [23]:
df_raw_2022_mn_pm25_rename = df_raw_2022_mn_pm25_dropped.rename(columns = columns)
df_raw_2022_mn_pm25_rename

,Date,POC,Day_Mean_PM25_Conc,Units,Daily_AQI,Site_Name,County,Latitude,Longitude
0,01/02/2022,1,13.5,ug/m3 LC,54,Anoka County Airport,Anoka,45.13768,-93.207615
1,01/05/2022,1,3.9,ug/m3 LC,16,Anoka County Airport,Anoka,45.13768,-93.207615
2,01/08/2022,1,9.6,ug/m3 LC,40,Anoka County Airport,Anoka,45.13768,-93.207615
3,01/11/2022,1,6.7,ug/m3 LC,28,Anoka County Airport,Anoka,45.13768,-93.207615
4,01/14/2022,1,2.6,ug/m3 LC,11,Anoka County Airport,Anoka,45.13768,-93.207615
...,...,...,...,...,...,...,...,...,...
8547,12/25/2022,3,3.6,ug/m3 LC,15,St. Michael Elementary School,Wright,45.20916,-93.669210
8548,12/26/2022,3,3.6,ug/m3 LC,15,St. Michael Elementary School,Wright,45.20916,-93.669210
8549,12/27/2022,3,5.0,ug/m3 LC,21,St. Michael Elementary School,Wright,45.20916,-93.669210
8550,12/28/2022,3,8.7,ug/m3 LC,36,St. Michael Elementary School,Wright,45.20916,-93.669210


In [ ]:
df_raw_2022_mn_pm25_rename["AQI_Concern"] = 